In [31]:
import os
os.chdir('/Users/zhanwenxin/Documents/GitHub/cuTAGI')

import numpy as np
from src.RL_functions.generate_one_synthetic_time_series import generate_one_synthetic_time_series
from scipy.stats import multivariate_normal
from src.RL_functions.actor_critic_kf import *
from tqdm import tqdm
from src.RL_functions.helpers import get_look_back_time_steps, hidden_states_collector

### Time series generation

### Analyze time series

In [32]:
# Time series config
components = ['level','ar']
time_step_interval = 1

AR_param_var = 5**2
AR_param_phi = 0.9
hyperparameters = {'level': {'process_error_var': 0.0},
                #    'fourrier': {'period': 52, 'process_error_var': 0},
                   'ar': {'phi': AR_param_phi, 'process_error_var': AR_param_var},
                   'observation': {'error': 1e-6}}
x_init = {'mu': np.array([5, -0.0621]), \
          'var': np.diag([0.00531, 6.36E-05])}
# x_init = {'mu': np.array([5, np.sqrt(AR_param_var) * 5, 0, -0.0621]), \
#           'var': np.diag([0.00531,1e-12, 1e-12, 6.36E-05])}
num_steps = 800
num_ts = 10


In [33]:
# Compute anomaly magnitude: proportional to the stationary variance of AR process
AR_stationary_var = hyperparameters['ar']['process_error_var'] / (1 - hyperparameters['ar']['phi']**2)

# Create a dictionary storing the data_generator.time_series['timesteps'], data_generator.time_series['y'], and anm_mag
data = {'timesteps': [], 'y': [], 'anm_mag': [], 'anm_pos': []}

# Generate time series
for ts_n in tqdm(range(num_ts)):
    np.random.seed(ts_n+100000)       # Seeds for testing
    # np.random.seed(ts_n)                # Seeds for training
    # # Fixed anomaly magnitude
    # anm_mag = - (np.sqrt(AR_stationary_var)*1) / 50
    # Random anm_mag between - (np.sqrt(AR_stationary_var)*1) / 25 and (np.sqrt(AR_stationary_var)*1) / 25
    anm_mag = np.random.uniform(- (np.sqrt(AR_stationary_var)*1) / 25, (np.sqrt(AR_stationary_var)*1) / 25)

    # Changed the AR each time
    data_generator = generate_one_synthetic_time_series(components = components, 
                                                        time_step_interval = time_step_interval, 
                                                        hyperparameters = hyperparameters, 
                                                        num_steps = num_steps,
                                                        x_init = x_init)

    # Periodic patten is fixed
    components_P = ['level','fourrier']
    hyperparameters_P = {'level': {'process_error_var': 0.0},
                    'fourrier': {'period': 52, 'process_error_var': 0},
                    'observation': {'error': 0}}
    x_init_P = {'mu': np.array([0, np.sqrt(AR_param_var) * 5, 0]), \
            'var': np.diag([0, 1e-12, 1e-12])}
    data_generator_P = generate_one_synthetic_time_series(components = components_P, 
                                                        time_step_interval = time_step_interval, 
                                                        hyperparameters = hyperparameters_P, 
                                                        num_steps = num_steps,
                                                        x_init = x_init_P)

    data_generator.time_series['y'] = (np.array(data_generator.time_series['y']) + np.array(data_generator_P.time_series['y'])).tolist()

    # Add anomaly
    anm_pos = 200
    for i in range(num_steps):
        if i >= anm_pos:
            data_generator.time_series['y'][i] += anm_mag * (i - anm_pos)
    # data_generator.plot()
    
    data['y'].append(data_generator.time_series['y'])
    data['anm_mag'].append(anm_mag)
    data['anm_pos'].append(anm_pos)

    # Store timesteps only once since they are the same for all time series
    if ts_n == 0:
        data['timesteps'].append(data_generator.time_series['timesteps'])
    else:
        data['timesteps'].append([])

# Save data as float number to 1 csv using pandas
import pandas as pd
data = pd.DataFrame(data)
data.to_csv('data/hsl_test_time_series_rand_anmmag_long.csv', index=False)


100%|██████████| 10/10 [00:02<00:00,  3.96it/s]


In [16]:
# Load time series, data as float
import pandas as pd
data = pd.read_csv('data/hsl_synthetic_time_series_rand_anmmag.csv')
data['y'] = data['y'].apply(lambda x: list(map(float, x[1:-1].split(','))))
# Convert data['timesteps'][0] from list of str to list of float
data['timesteps'][0] = list(map(float, data['timesteps'][0][1:-1].split(',')))
data['anm_mag'] = data['anm_mag'].apply(lambda x: float(x))
data['anm_pos'] = data['anm_pos'].apply(lambda x: int(x))

/var/folders/vv/62ff9wx97g72zr4_20vx1q3h0000gn/T/ipykernel_97886/1472982120.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['timesteps'][0] = list(map(float, data['timesteps'][0][1:-1].split(',')))


In [17]:
# # Config for the base and drift models
components = ['trend','fourrier', 'ar']
components_d = ['trend', 'ar']

hyperparameters = {'trend': {'process_error_var': 0},
                    'fourrier': {'period': 52, 'process_error_var': 0},
                    'ar': {'phi': AR_param_phi, 'process_error_var': AR_param_var},
                    'observation': {'error': 1e-6}}
hyperparameters_d = {'trend': {'process_error_var': 0, 'phi': 1},
                    'ar': {'phi': AR_param_phi, 'process_error_var': AR_param_var},
                    'observation': {'error': 1e-6}}

x_init = {'mu': np.array([5, 0, np.sqrt(AR_param_var) * 5, 0, -0.0621]), \
            'var': np.diag([1e-12, 1E-12, 1e-12, 1E-12, 6.36E-05])}
x_init_d = {'mu': np.array([0, 0.0, -0.0621]), \
            'var': np.diag([1e-12, 1e-12, 6.36E-05])}

baseline_process_error_var_all = [1e-6]

In [18]:
# kf = KalmanFilter(components = components, time_step=time_step_interval, hyperparameters = hyperparameters)
# x_samples_mean_all, x_samples_cov_all = [], []
# for j, baseline_process_error_var in enumerate(baseline_process_error_var_all):
#     hyperparameters_d['trend']['process_error_var'] = baseline_process_error_var
#     kf_d = KalmanFilter(components = components_d, time_step=time_step_interval, hyperparameters = hyperparameters_d)
#     x_samples_mean, x_samples_cov = estimate_hs_distribution(components, time_step_interval, hyperparameters, num_steps, x_init, x_init_d, kf, kf_d)
#     x_samples_mean_all.append(x_samples_mean)
#     x_samples_cov_all.append(x_samples_cov)
#     print(x_samples_mean, x_samples_cov)

# Hidden state distribution provided by users
x_samples_mean_all = [-0.00080167]
x_samples_cov_all = [2.0644215838756797e-05]

[-0.00080167] 2.0644215838756797e-05


In [7]:
# Analyze the time series using base and drift models
samples = {'LTd_history': [], 'itv_LT': [], 'itv_LL': [], 'anm_develop_time': []}

kf = KalmanFilter(components = components, time_step=time_step_interval, hyperparameters = hyperparameters)
prior_a = [0.998, 0.002]

mv_normal_x_all = []
scale_factors_all = []
for j, baseline_process_error_var in enumerate(baseline_process_error_var_all):
    x_samples_mean, x_samples_cov = x_samples_mean_all[j], x_samples_cov_all[j]
    mv_normal_x = multivariate_normal(mean=x_samples_mean, cov=x_samples_cov, allow_singular=True)
    mv_normal_x_all.append(mv_normal_x)
    scale_factors_all.append(mv_normal_x.pdf(x_samples_mean))

# for ts_n in range(2):
for ts_n in tqdm(range(len(data['y']))):

    x_last_step = x_init
    x_last_step_d = x_init_d

    intervene = False
    intervention_state = None
    stop_collecting_samples = False

    p_na_I_Mj_Yt1 = prior_a[0]
    p_a_I_Mj_Yt1 = prior_a[1]
    p_Mj_I_Yt_all = []
    p_a0_all = []
    LTd_all = {'mu': [], 'var': []}
    plot_model_index = 0
    for i in range(num_steps):
        xd_updated_remain_all = []
        x_y_likelihood = []
        for j, baseline_process_error_var in enumerate(baseline_process_error_var_all):
            hyperparameters_d['trend']['process_error_var'] = baseline_process_error_var
            kf_d = KalmanFilter(components = components_d, time_step=time_step_interval, hyperparameters = hyperparameters_d)
            x_last_step_d = x_init_d if i==0 else x_last_step_d_all[j]

            intervention_state = copy.deepcopy(x_last_step)
            intervention_state['mu'][0] += x_last_step_d['mu'][0]
            intervention_state['mu'][1] += x_last_step_d['mu'][1]
            intervention_state['var'][0,0] += x_last_step_d['var'][0,0]
            intervention_state['var'][1,1] += x_last_step_d['var'][1,1]
            intervention_state['mu'][-1] = x_last_step_d['mu'][-1]
            intervention_state['var'][-1,-1] = x_last_step_d['var'][-1,-1]

            if j==plot_model_index:
                y_likelihood_a, y_likelihood_na, x_likelihood_a, x_likelihood_na,\
                    _, xd_updated_remain, _, _, x_pred, xd_pred, _, yj_pred, _, ar_pred = critic(kf, kf_d, x_last_step, x_last_step_d, intervention_state, x_init_d, data['y'][ts_n][i], mv_normal_x_all[j])
            else:
                y_likelihood_a, y_likelihood_na, x_likelihood_a, x_likelihood_na,\
                    _, xd_updated_remain, _, _, _, xd_pred, _, _, _, _ = critic(kf, kf_d, x_last_step, x_last_step_d, intervention_state, x_init_d, data['y'][ts_n][i], mv_normal_x_all[j])
            x_likelihood_a *= 1/scale_factors_all[j]
            x_likelihood_na *= 1/scale_factors_all[j]

            x_y_likelihood.append([x_likelihood_na.item(), x_likelihood_a.item(), y_likelihood_na.item(), y_likelihood_a.item()])
            xd_updated_remain_all.append(xd_updated_remain)

        x_y_likelihood_array = np.array(x_y_likelihood)
        p_yt_I_Mj_Yt1 = x_y_likelihood_array[:, 2] * x_y_likelihood_array[:, 0] * p_na_I_Mj_Yt1 + x_y_likelihood_array[:, 3] * x_y_likelihood_array[:, 1] * p_a_I_Mj_Yt1
        p_na_I_Mj_Yt = x_y_likelihood_array[:, 2] * x_y_likelihood_array[:, 0] * p_na_I_Mj_Yt1 / p_yt_I_Mj_Yt1
        p_a_I_Mj_Yt = x_y_likelihood_array[:, 3] * x_y_likelihood_array[:, 1] * p_a_I_Mj_Yt1 / p_yt_I_Mj_Yt1

        # Compute probability of taking intervention or not
        p_Mj_I_Yt = p_yt_I_Mj_Yt1 / np.sum(p_yt_I_Mj_Yt1)   # Probability of each model
        p_a0 = np.sum(p_na_I_Mj_Yt * p_Mj_I_Yt)             # Probability of not requiring intervention by all models

        p_Mj_I_Yt_all.append(p_Mj_I_Yt.tolist())
        p_a0_all.append(p_a0)

        if i > data['anm_pos'][ts_n] and stop_collecting_samples is not True:
            # Look back history for LTd
            LTd_history = hidden_states_collector(i - 1, LTd_all)
            samples['LTd_history'].append(LTd_history['mu'].tolist())  # TODO
            samples['itv_LT'].append(data['anm_mag'][ts_n])
            anm_develop_time = i - 1 - data['anm_pos'][ts_n]
            samples['anm_develop_time'].append(anm_develop_time)
            samples['itv_LL'].append(data['anm_mag'][ts_n] * anm_develop_time)

        if p_a0 <= 0.5:
            stop_collecting_samples = True
        
        if stop_collecting_samples is True:
            break

        x_pred, y_pred, x_last_step = actor(kf, x_last_step, intervene, intervention_state, data['y'][ts_n][i])

        # Store all LTd for history collection
        LTd_all['mu'].append(xd_pred['mu'][1])
        LTd_all['var'].append(xd_pred['var'][1,1])

        x_last_step_d_all = xd_updated_remain_all


100%|██████████| 1000/1000 [03:48<00:00,  4.37it/s]


In [8]:
# Save samples in a csv
samples_df = pd.DataFrame(samples)
samples_df.to_csv('data/hsl_synthetic_samples_rand_anmmag.csv', index=False)

In [9]:
print(len(samples['LTd_history']))
print(len(samples['itv_LT']))
print(len(samples['itv_LL']))
print(len(samples['anm_develop_time']))
print(samples['LTd_history'][97])
print(samples['LTd_history'][207])
print(samples['anm_develop_time'])

111826
111826
111826
111826
[-0.01133922666698772, -0.010490440889140664, -0.0103439888112002, -0.008483388065740127, -0.008273387123530542, -0.008300419704727072, -0.005637479444361801, -0.004300724422254199]
[-0.003025580197822505, -0.0026969509382484924, -0.0028599797137009693, -0.0026846185853906376, -0.0020973227151069545, -0.0021506079154607004, 0.00013300880651384453, 0.0008850087923096123]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 